In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from modules.dataset_loader import DatasetLoader
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical



/home/gabriel/.pyenv/versions/machine-learning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/gabriel/.pyenv/versions/machine-learning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/gabriel/.pyenv/versions/machine-learning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/gabr

In [2]:
def add_padding(X,y):
    X = np.asarray(X)
    y = np.asarray(y)
    
    max_len = len(X[0])
    for row in X:
        if len(row) > max_len:
            max_len = len(row)

    X = pad_sequences(X, maxlen=max_len, padding='post', dtype='float64')
    return X, y


def load_dataset():
    DATASET_PATH = 'datasets/Original/MFCC/'
    dataset_loader = DatasetLoader(DATASET_PATH)
    mfcc_features, y = dataset_loader.get_dataset()

    DATASET_PATH = 'datasets/Original/Prosody/'
    dataset_loader = DatasetLoader(DATASET_PATH)
    prosody_features, y = dataset_loader.get_dataset()


    new_dataset = []
    for index in range(0, len(mfcc_features)):
        new_instance = []
        for row_index in range(0, len(mfcc_features[index])):
            new_row = np.concatenate(
                (mfcc_features[index][row_index],
                prosody_features[index][row_index]),
                axis= None
            )
            new_instance.append(new_row)
        new_dataset.append(new_instance)

    X = new_dataset

    DATASET_PATH = 'datasets/Original/Chroma/'
    dataset_loader = DatasetLoader(DATASET_PATH)
    chroma_features, y = dataset_loader.get_dataset()

    new_dataset = []
    for index in range(0, len(chroma_features)):
        new_instance = []
        for row_index in range(0, len(chroma_features[index])):
            new_row = np.concatenate(
                (X[index][row_index],
                chroma_features[index][row_index]),
                axis= None
            )
            new_instance.append(new_row)
        new_dataset.append(new_instance)

    X = np.asarray(new_dataset)
    
    return X, y

In [3]:
models_file = [] 
results_dir = os.listdir('best_hyperopt_results')
for filename in results_dir:
    if '.h5' in filename:
        models_file.append("best_hyperopt_results/" + filename)

In [4]:
models_file

['best_hyperopt_results/tri.h5',
 'best_hyperopt_results/des.h5',
 'best_hyperopt_results/ale.h5',
 'best_hyperopt_results/sur.h5',
 'best_hyperopt_results/rai.h5',
 'best_hyperopt_results/neu.h5',
 'best_hyperopt_results/med.h5']

In [5]:
models = dict()
for model_file in models_file:
    models[model_file.split('/')[-1][:-3]] = keras.models.load_model(model_file)


In [6]:
models

{'tri': <tensorflow.python.keras.engine.sequential.Sequential at 0x7feac656e630>,
 'des': <tensorflow.python.keras.engine.sequential.Sequential at 0x7feac6554eb8>,
 'ale': <tensorflow.python.keras.engine.sequential.Sequential at 0x7feac656e080>,
 'sur': <tensorflow.python.keras.engine.sequential.Sequential at 0x7feac5c2fe80>,
 'rai': <tensorflow.python.keras.engine.sequential.Sequential at 0x7feabc2b4ef0>,
 'neu': <tensorflow.python.keras.engine.sequential.Sequential at 0x7feabc21f588>,
 'med': <tensorflow.python.keras.engine.sequential.Sequential at 0x7fea94751dd8>}

In [7]:
dataset_loader = DatasetLoader()
X,y = dataset_loader.compose_complete_dataset()

In [11]:
y = [inst[0] for inst in y]

In [13]:
X, y = add_padding(X,y)
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3)


y_test_labels = y_test
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

num_rows = X[0].shape[0]
num_columns = X[0].shape[1]
num_channels = 1
X_train = X_train.reshape(X_train.shape[0], num_rows, num_columns,num_channels)
X_test = X_test.reshape(X_test.shape[0], num_rows, num_columns,num_channels)

In [17]:
x0 = X_train[0]

In [18]:
x0

array([[[ 2.        ],
        [ 0.02      ],
        [11.58433   ],
        ...,
        [ 0.0680059 ],
        [ 0.07016346],
        [ 0.07605148]],

       [[ 3.        ],
        [ 0.03      ],
        [13.21423   ],
        ...,
        [ 0.04753622],
        [ 0.05832832],
        [ 0.09664869]],

       [[ 4.        ],
        [ 0.04      ],
        [11.5333    ],
        ...,
        [ 0.0298175 ],
        [ 0.03380428],
        [ 0.0964592 ]],

       ...,

       [[ 0.        ],
        [ 0.        ],
        [ 0.        ],
        ...,
        [ 0.        ],
        [ 0.        ],
        [ 0.        ]],

       [[ 0.        ],
        [ 0.        ],
        [ 0.        ],
        ...,
        [ 0.        ],
        [ 0.        ],
        [ 0.        ]],

       [[ 0.        ],
        [ 0.        ],
        [ 0.        ],
        ...,
        [ 0.        ],
        [ 0.        ],
        [ 0.        ]]])

In [23]:
models['tri'].predict(X_train)

ValueError: Error when checking input: expected conv2d_1_input to have shape (485, 36, 1) but got array with shape (540, 36, 1)

In [27]:
X_train[0].shape

(540, 36, 1)